# 数据合并处理

In [11]:
import pandas as pd
import numpy as np
import scipy as sp


def read_csv_file(f, logging=False):
    data = pd.read_csv(f)
    if logging:
        print(data.head())
        print(data.columns.values)
        print(data.describe())
        print(data.info())
    return data


def categories_process_first_class(cate):
    cate = str(cate)
    return int(cate[0])


def categories_process_second_class(cate):
    cate = str(cate)
    if len(cate) < 3:
        return 0
    else:
        return int(cate[1:])


def age_process(age):
    age = int(age)
    if age == 0:
        return 0
    if age < 16:
        return 1
    if age < 23:
        return 2
    if age < 27:
        return 3
    else:
        return 4


def province_process(hometown):
    hometown = str(hometown)
    if len(hometown) == 1:
        return 0
    else:
        return int(hometown[:-2])


def city_process(hometown):
    hometown = str(hometown)
    if len(hometown) == 1:
        return 0
    else:
        return int(hometown[-2:])


def get_time_day(t):
    t = str(t)
    t = int(t[0:2])
    return t


def get_time_hour(t):
    t = str(t)
    t = int(t[2:4])
    if t < 6:
        return 0
    if t < 12:
        return 1
    if t < 18:
        return 2
    else:
        return 3


def get_time_min(t):
    t = str(t)
    t = int(t[4:])
    return t


def logloss(act, pred):
    epsilon = 1e-5
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1 - eplison, pred)
    ll = sum(act * sp.log(pred) +
             sp.subtract(1, act) * sp.log(sp.subtract(1, pred)))
    ll = ll * -1.0 / len(act)
    return ll

In [12]:
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

print('test dataset')
test_data = read_csv_file('./pre/test.csv', logging=True)
print('================================================\n')
print('train dataset:')
train_data = read_csv_file('./pre/train.csv', logging=True)
print('================================================\n')
print('ad dataset')
ad = read_csv_file('./pre/ad.csv', logging=True)
print('================================================\n')
print('app dataset')
app_categories = read_csv_file('./pre/app_categories.csv', logging=True)
print('================================================\n')
print('user dataset')
user = read_csv_file('./pre/user.csv', logging=True)
print('================================================\n')
print('position dataset')
position = read_csv_file('./pre/position.csv', logging=True)
print('================================================\n')
print('user_app_actions dataset')
user_app_actions = read_csv_file('./pre/user_app_actions.csv', logging=True)

test dataset
   instanceID  label  clickTime  creativeID   userID  positionID  \
0           1     -1     310000        3745  1164848        3451   
1           2     -1     310000        2284  2127247        1613   
2           3     -1     310000        1456  2769125        5510   
3           4     -1     310000        4565     9762        4113   
4           5     -1     310000          49  2513636        3615   

   connectionType  telecomsOperator  
0               1                 3  
1               1                 3  
2               2                 1  
3               2                 3  
4               1                 3  
['instanceID' 'label' 'clickTime' 'creativeID' 'userID' 'positionID'
 'connectionType' 'telecomsOperator']
          instanceID     label      clickTime     creativeID        userID  \
count  338489.000000  338489.0  338489.000000  338489.000000  3.384890e+05   
mean   169245.000000      -1.0  311479.490613    3001.534765  1.409519e+06   
std     9

   userID  installTime  appID
0       1       182032    933
1       5       141354    589
2       5       251727    565
3       5       260932    596
4       5       261318    536
['userID' 'installTime' 'appID']
             userID   installTime         appID
count  6.003471e+06  6.003471e+06  6.003471e+06
mean   1.403576e+06  1.483695e+05  5.125716e+03
std    8.074011e+05  9.097316e+04  2.118739e+04
min    1.000000e+00  1.000000e+04  3.540000e+02
25%    7.055445e+05  6.164500e+04  5.890000e+02
50%    1.404560e+06  1.408000e+05  8.270000e+02
75%    2.100952e+06  2.320160e+05  1.969000e+03
max    2.805117e+06  3.023590e+05  4.332670e+05
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6003471 entries, 0 to 6003470
Data columns (total 3 columns):
userID         int64
installTime    int64
appID          int64
dtypes: int64(3)
memory usage: 137.4 MB
None


In [ ]:
print('================================================\n')
print('user_installedapps dataset')
user_installedapps = read_csv_file('./pre/user_installedapps.csv',
                                   logging=False)

In [ ]:
train = train_data.drop(['label', 'conversionTime'], axis=1)
test = test_data.drop(['label', 'instanceID'], axis=1)
data = pd.concat((train, test), axis=0)

In [ ]:
print(data.shape)
print(train.shape)
print(test.shape)

In [ ]:
data.isnull().sum()

In [ ]:
data_ad = data.merge(ad, on='creativeID', how='left')
data_ad.shape

In [ ]:
data_ad['click_day'] = data_ad['clickTime'].apply(get_time_day)
data_ad['click_min'] = data_ad['clickTime'].apply(get_time_min)
data_ad['click_hour'] = data_ad['clickTime'].apply(get_time_hour)
data_ad.drop(['clickTime'], inplace=True, axis=1)
data_ad.tail()

In [ ]:
data_ad_app = data_ad.merge(app_categories, on='appID', how='left')
data_ad_app['app_first_categories'] = data_ad_app['appCategory'].apply(
    categories_process_first_class)
data_ad_app['app_second_categories'] = data_ad_app['appCategory'].apply(
    categories_process_second_class)
data_ad_app.drop(['appCategory'], axis=1, inplace=True)

In [ ]:
data_ad_app_user = data_ad_app.merge(user, on='userID', how='left')

In [ ]:
##test数据集的age数据缺失较多
data_ad_app_user['age'].describe()

In [ ]:
data_ad_app_user['age'].replace(
    0.0,
    data_ad_app_user['age'][data_ad_app_user['age'] != 0.0].mean(),
    inplace=True)

In [ ]:
data_ad_app_user['age'].fillna(
    data_ad_app_user['age'][data_ad_app_user['age'] != 0.0].mean(),
    inplace=True)
data_ad_app_user['age'].isnull().sum()

In [ ]:
data_ad_app_user['age'] = data_ad_app_user['age'].apply(age_process)

In [ ]:
data_ad_app_user['residence_province'] = data_ad_app_user['residence'].apply(
    province_process)
data_ad_app_user['residence_city'] = data_ad_app_user['residence'].apply(
    city_process)
data_ad_app_user.drop(['residence'], inplace=True, axis=1)
data_ad_app_user.tail()

In [ ]:
data_ad_app_user_position = data_ad_app_user.merge(position,
                                                   on='positionID',
                                                   how='left')

In [ ]:
x_train = data_ad_app_user_position[:3749528]
y_train = train_data['label']
test = data_ad_app_user_position[3749528:]

In [ ]:
import pickle
with open('values.pkl', 'wb') as f:
    pickle.dump(x_train, f)
    pickle.dump(y_train, f)
    pickle.dump(test, f)

# 数据建模

## 导入数据

In [22]:
import pickle
with open('values.pkl', 'rb') as f:
    x_train = pickle.load(f)
    y_train = pickle.load(f)
    test = pickle.load(f)

In [23]:
y_train.value_counts()
##正负样本非常不均衡，但由于过采样后数据量过大，电脑内存有限，使用欠采样的方法

0    3656266
1      93262
Name: label, dtype: int64

In [4]:
x_train.head()

,creativeID,userID,positionID,connectionType,telecomsOperator,adID,camgaignID,advertiserID,appID,appPlatform,...,gender,education,marriageStatus,haveBaby,hometown_province,hometown_city,residence_province,residence_city,sitesetID,positionType
0,3089,2798058,293,1,1,1321,83,10,434,1,...,2,1,3,1,0,0,13,1,0,3
1,1259,463234,6161,1,2,1535,685,80,14,2,...,2,1,1,0,0,0,2,13,0,1
2,4465,1857485,7434,4,1,147,460,3,465,1,...,1,1,0,0,0,0,15,2,0,1
3,1004,2038823,977,1,1,411,564,3,465,1,...,0,0,0,0,0,0,10,1,0,1
4,1887,2015141,3688,1,1,369,144,84,360,1,...,1,2,2,0,10,1,10,1,0,1


In [12]:
x_train[[
    'click_day', 'click_min', 'click_hour', 'app_first_categories',
    'app_second_categories', 'age'
]].head()

,click_day,click_min,click_hour,app_first_categories,app_second_categories,age
0,17,0,0,1,8,3
1,17,0,0,2,0,2
2,17,0,0,2,9,2
3,17,0,0,2,9,0
4,17,0,0,2,1,4


## 过采样后存储数据

In [ ]:
from imblearn.over_sampling import ADASYN

x_train_oversam, y_train_oversam = ADASYN().fit_sample(x_train, y_train)

In [ ]:
import pickle
with open('values_resam.pkl', 'wb') as f:
    pickle.dump(x_train_oversam, f)
    pickle.dump(y_train_oversam, f)

In [ ]:
print(x_train_oversam.shape)
print(y_train_oversam.shape)

In [ ]:
## EasyEnsemble
'''
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold

bbc = BalancedBaggingClassifier(
    base_estimator = DecisionTreeClassifier(),
    ratio = 'auto',
    replacement = False,
    random_state = 100,
    n_jobs = -1
)'''
'''
kf = KFold(n_splits = 5)
for train_index, val_index in kf.split(x_train, y_train):
    X_train,Y_train = x_train[train_index, : ], y_train[train_index]
    X_val, Y_val = x_train[val_index, : ], y_train[val_index]
    bbc.fit(X_train, Y_train)
    pred_bbc = bbc.predict_proba(X_val)
    print(pred_bbc)
    print('The logloss is ', logloss(Y_val, pred_bbc))
'''

## 欠采样并存储数据

In [ ]:
import numpy as np
import pickle
from imblearn.under_sampling import RandomUnderSampler

with open('values.pkl', 'rb') as f:
    x_train = pickle.load(f)
    y_train = pickle.load(f)
    test = pickle.load(f)

#x_train = np.array(x_train)
#y_train = np.array(y_train)
#test = np.array(test)

In [18]:
y_train = train_data['label']
y_train = pd.DataFrame(y_train, columns=['label'])

In [21]:
from imblearn.under_sampling import RandomUnderSampler

bc = RandomUnderSampler(random_state=100)
x_train_resam, y_train_resam = bc.fit_sample(x_train, y_train)

with open('values_undersampling.pkl', 'wb') as f:
    pickle.dump(x_train_resam, f)
    pickle.dump(y_train_resam, f)
    pickle.dump(test, f)

In [ ]:
x_train_resam = pd.DataFrame(x_train_resam, columns=x_train.columns)
y_train_resam = pd.DataFrame(y_train_resam, columns=['label'])
print(x_train_resam)

import pickle

with open('lgb_values.pkl', 'wb') as f:
    pickle.dump(x_train_resam, f)
    pickle.dump(y_train_resam, f)
    pickle.dump(test, f)

## 载入数据

In [2]:
import pickle

with open('values_undersampling.pkl', 'rb') as f:
    x_train_resam = pickle.load(f)
    y_train_resam = pickle.load(f)
    test = pickle.load(f)

### Xgbosst

#### 用sklearn API

In [ ]:
'''
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid = {
    'max_depth': [3, 4, 5, 8, 10],
    'n_estimators': [50, 100, 200, 400, 600, 800, 1000],
    'laerning_rate': [0.1, 0.2, 0.3],
    'gamma': [0, 0.2],
    'subsample': [0.8, 1],
             }

xgb_model = xgb.XGBClassifier()
rgs = GridSearchCV(xgb_model, param_grid, n_jobs = -1, cv = 5)
rgs.fit(x_train_resam, y_train_resam)
print(rgs.best_score_)
print(rgs.best_params_)
pred = rgs.predict_proba(test)
print('The logloss is ', logloss(Y_val, pred_bbc))
'''

#### 用Python API

In [ ]:
'''
import xgboost as xgb

dtrain = xgb.DMatrix(x_train_resam, label = y_train_resam)
num_xgb = 500

params = {
    'booster': 'gbtree',
    'eta': 0.2, 
    'eval_metric': 'logloss',
    'tree method': 'auto',
    'objective': 'binary:logistic'
}

bst = xgb.cv(params, dtrain, num_xgb, nfold = 5, verbose_eval = True)

dtest = xgb.DMatrix(test)
pred = bst.predict(dtest)
print(logloss()
'''

### LightGBM

In [3]:
import pickle

with open('lgb_values.pkl', 'rb') as f:
    x_train_resam = pickle.load(f)
    y_train_resam = pickle.load(f)
    test = pickle.load(f)

In [4]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_val, y_train, y_val = train_test_split(x_train_resam,
                                                  y_train_resam,
                                                  test_size=0.2)
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 3,
    'max_depth': 10,
    'min_data_in_leaf': 40,
    'learning_rate': 0.5,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'lambda_l2': 0.001,
    'verbose': 1
}

lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=2000,
                verbose_eval=True,
                valid_sets=lgb_val,
                early_stopping_rounds=None)
pred = gbm.predict(test, num_iteration=gbm.best_iteration)

importance = gbm.feature_importance()
names = gbm.feature_name()
for idx, im in sorted(enumerate(importance), key=lambda x: x[1], reverse=True):
    print('The score of {} is {}'.format(names[idx], im))

[1]	valid_0's binary_logloss: 0.663541
[2]	valid_0's binary_logloss: 0.649042
[3]	valid_0's binary_logloss: 0.64208
[4]	valid_0's binary_logloss: 0.636095
[5]	valid_0's binary_logloss: 0.632475
[6]	valid_0's binary_logloss: 0.628151
[7]	valid_0's binary_logloss: 0.625938
[8]	valid_0's binary_logloss: 0.62288
[9]	valid_0's binary_logloss: 0.620696
[10]	valid_0's binary_logloss: 0.619348
[11]	valid_0's binary_logloss: 0.618298
[12]	valid_0's binary_logloss: 0.616979
[13]	valid_0's binary_logloss: 0.614798
[14]	valid_0's binary_logloss: 0.613518
[15]	valid_0's binary_logloss: 0.611541
[16]	valid_0's binary_logloss: 0.610836
[17]	valid_0's binary_logloss: 0.610136
[18]	valid_0's binary_logloss: 0.609631
[19]	valid_0's binary_logloss: 0.608989
[20]	valid_0's binary_logloss: 0.60836
[21]	valid_0's binary_logloss: 0.607665
[22]	valid_0's binary_logloss: 0.60719
[23]	valid_0's binary_logloss: 0.606441
[24]	valid_0's binary_logloss: 0.605922
[25]	valid_0's binary_logloss: 0.605301
[26]	valid_0'

[205]	valid_0's binary_logloss: 0.570002
[206]	valid_0's binary_logloss: 0.569867
[207]	valid_0's binary_logloss: 0.56973
[208]	valid_0's binary_logloss: 0.569739
[209]	valid_0's binary_logloss: 0.569673
[210]	valid_0's binary_logloss: 0.569596
[211]	valid_0's binary_logloss: 0.569555
[212]	valid_0's binary_logloss: 0.569572
[213]	valid_0's binary_logloss: 0.569504
[214]	valid_0's binary_logloss: 0.569505
[215]	valid_0's binary_logloss: 0.569485
[216]	valid_0's binary_logloss: 0.569462
[217]	valid_0's binary_logloss: 0.569463
[218]	valid_0's binary_logloss: 0.569318
[219]	valid_0's binary_logloss: 0.569268
[220]	valid_0's binary_logloss: 0.569232
[221]	valid_0's binary_logloss: 0.569202
[222]	valid_0's binary_logloss: 0.569099
[223]	valid_0's binary_logloss: 0.569079
[224]	valid_0's binary_logloss: 0.56906
[225]	valid_0's binary_logloss: 0.569034
[226]	valid_0's binary_logloss: 0.569025
[227]	valid_0's binary_logloss: 0.569018
[228]	valid_0's binary_logloss: 0.569037
[229]	valid_0's bi

[418]	valid_0's binary_logloss: 0.564323
[419]	valid_0's binary_logloss: 0.564311
[420]	valid_0's binary_logloss: 0.56429
[421]	valid_0's binary_logloss: 0.56425
[422]	valid_0's binary_logloss: 0.56423
[423]	valid_0's binary_logloss: 0.564255
[424]	valid_0's binary_logloss: 0.564177
[425]	valid_0's binary_logloss: 0.564153
[426]	valid_0's binary_logloss: 0.564173
[427]	valid_0's binary_logloss: 0.56418
[428]	valid_0's binary_logloss: 0.564124
[429]	valid_0's binary_logloss: 0.564096
[430]	valid_0's binary_logloss: 0.564062
[431]	valid_0's binary_logloss: 0.564041
[432]	valid_0's binary_logloss: 0.564063
[433]	valid_0's binary_logloss: 0.563935
[434]	valid_0's binary_logloss: 0.563938
[435]	valid_0's binary_logloss: 0.563936
[436]	valid_0's binary_logloss: 0.563912
[437]	valid_0's binary_logloss: 0.563893
[438]	valid_0's binary_logloss: 0.563978
[439]	valid_0's binary_logloss: 0.563975
[440]	valid_0's binary_logloss: 0.563907
[441]	valid_0's binary_logloss: 0.563951
[442]	valid_0's bina

[636]	valid_0's binary_logloss: 0.561902
[637]	valid_0's binary_logloss: 0.561881
[638]	valid_0's binary_logloss: 0.561894
[639]	valid_0's binary_logloss: 0.561831
[640]	valid_0's binary_logloss: 0.561825
[641]	valid_0's binary_logloss: 0.561806
[642]	valid_0's binary_logloss: 0.561776
[643]	valid_0's binary_logloss: 0.561738
[644]	valid_0's binary_logloss: 0.561687
[645]	valid_0's binary_logloss: 0.561657
[646]	valid_0's binary_logloss: 0.561655
[647]	valid_0's binary_logloss: 0.561653
[648]	valid_0's binary_logloss: 0.56164
[649]	valid_0's binary_logloss: 0.561614
[650]	valid_0's binary_logloss: 0.56164
[651]	valid_0's binary_logloss: 0.56167
[652]	valid_0's binary_logloss: 0.561644
[653]	valid_0's binary_logloss: 0.561657
[654]	valid_0's binary_logloss: 0.561654
[655]	valid_0's binary_logloss: 0.561605
[656]	valid_0's binary_logloss: 0.561545
[657]	valid_0's binary_logloss: 0.561537
[658]	valid_0's binary_logloss: 0.561515
[659]	valid_0's binary_logloss: 0.561451
[660]	valid_0's bin

[838]	valid_0's binary_logloss: 0.56126
[839]	valid_0's binary_logloss: 0.561248
[840]	valid_0's binary_logloss: 0.561239
[841]	valid_0's binary_logloss: 0.561222
[842]	valid_0's binary_logloss: 0.561219
[843]	valid_0's binary_logloss: 0.561229
[844]	valid_0's binary_logloss: 0.56118
[845]	valid_0's binary_logloss: 0.561195
[846]	valid_0's binary_logloss: 0.561164
[847]	valid_0's binary_logloss: 0.561137
[848]	valid_0's binary_logloss: 0.56114
[849]	valid_0's binary_logloss: 0.561127
[850]	valid_0's binary_logloss: 0.561114
[851]	valid_0's binary_logloss: 0.561153
[852]	valid_0's binary_logloss: 0.561128
[853]	valid_0's binary_logloss: 0.561069
[854]	valid_0's binary_logloss: 0.561086
[855]	valid_0's binary_logloss: 0.561107
[856]	valid_0's binary_logloss: 0.561094
[857]	valid_0's binary_logloss: 0.561102
[858]	valid_0's binary_logloss: 0.561078
[859]	valid_0's binary_logloss: 0.561082
[860]	valid_0's binary_logloss: 0.561029
[861]	valid_0's binary_logloss: 0.560998
[862]	valid_0's bin

[1047]	valid_0's binary_logloss: 0.560528
[1048]	valid_0's binary_logloss: 0.560542
[1049]	valid_0's binary_logloss: 0.560529
[1050]	valid_0's binary_logloss: 0.560571
[1051]	valid_0's binary_logloss: 0.560555
[1052]	valid_0's binary_logloss: 0.560562
[1053]	valid_0's binary_logloss: 0.560559
[1054]	valid_0's binary_logloss: 0.560565
[1055]	valid_0's binary_logloss: 0.560569
[1056]	valid_0's binary_logloss: 0.560558
[1057]	valid_0's binary_logloss: 0.560522
[1058]	valid_0's binary_logloss: 0.56056
[1059]	valid_0's binary_logloss: 0.560543
[1060]	valid_0's binary_logloss: 0.560511
[1061]	valid_0's binary_logloss: 0.560454
[1062]	valid_0's binary_logloss: 0.560437
[1063]	valid_0's binary_logloss: 0.560462
[1064]	valid_0's binary_logloss: 0.560445
[1065]	valid_0's binary_logloss: 0.560459
[1066]	valid_0's binary_logloss: 0.560448
[1067]	valid_0's binary_logloss: 0.560451
[1068]	valid_0's binary_logloss: 0.560429
[1069]	valid_0's binary_logloss: 0.560418
[1070]	valid_0's binary_logloss: 0.

[1255]	valid_0's binary_logloss: 0.560389
[1256]	valid_0's binary_logloss: 0.560411
[1257]	valid_0's binary_logloss: 0.560389
[1258]	valid_0's binary_logloss: 0.560443
[1259]	valid_0's binary_logloss: 0.560421
[1260]	valid_0's binary_logloss: 0.560434
[1261]	valid_0's binary_logloss: 0.560421
[1262]	valid_0's binary_logloss: 0.560498
[1263]	valid_0's binary_logloss: 0.560478
[1264]	valid_0's binary_logloss: 0.560501
[1265]	valid_0's binary_logloss: 0.560527
[1266]	valid_0's binary_logloss: 0.560538
[1267]	valid_0's binary_logloss: 0.56053
[1268]	valid_0's binary_logloss: 0.56053
[1269]	valid_0's binary_logloss: 0.560542
[1270]	valid_0's binary_logloss: 0.560556
[1271]	valid_0's binary_logloss: 0.560559
[1272]	valid_0's binary_logloss: 0.560556
[1273]	valid_0's binary_logloss: 0.560575
[1274]	valid_0's binary_logloss: 0.560563
[1275]	valid_0's binary_logloss: 0.560555
[1276]	valid_0's binary_logloss: 0.560547
[1277]	valid_0's binary_logloss: 0.560547
[1278]	valid_0's binary_logloss: 0.5

[1451]	valid_0's binary_logloss: 0.560894
[1452]	valid_0's binary_logloss: 0.560904
[1453]	valid_0's binary_logloss: 0.560924
[1454]	valid_0's binary_logloss: 0.56088
[1455]	valid_0's binary_logloss: 0.56084
[1456]	valid_0's binary_logloss: 0.56082
[1457]	valid_0's binary_logloss: 0.560809
[1458]	valid_0's binary_logloss: 0.560779
[1459]	valid_0's binary_logloss: 0.560773
[1460]	valid_0's binary_logloss: 0.5608
[1461]	valid_0's binary_logloss: 0.5608
[1462]	valid_0's binary_logloss: 0.560828
[1463]	valid_0's binary_logloss: 0.560877
[1464]	valid_0's binary_logloss: 0.560882
[1465]	valid_0's binary_logloss: 0.560856
[1466]	valid_0's binary_logloss: 0.560874
[1467]	valid_0's binary_logloss: 0.560877
[1468]	valid_0's binary_logloss: 0.560865
[1469]	valid_0's binary_logloss: 0.560852
[1470]	valid_0's binary_logloss: 0.560765
[1471]	valid_0's binary_logloss: 0.560761
[1472]	valid_0's binary_logloss: 0.560736
[1473]	valid_0's binary_logloss: 0.56074
[1474]	valid_0's binary_logloss: 0.560734


[1656]	valid_0's binary_logloss: 0.560687
[1657]	valid_0's binary_logloss: 0.560714
[1658]	valid_0's binary_logloss: 0.560733
[1659]	valid_0's binary_logloss: 0.560688
[1660]	valid_0's binary_logloss: 0.560703
[1661]	valid_0's binary_logloss: 0.560693
[1662]	valid_0's binary_logloss: 0.560697
[1663]	valid_0's binary_logloss: 0.560714
[1664]	valid_0's binary_logloss: 0.560702
[1665]	valid_0's binary_logloss: 0.560724
[1666]	valid_0's binary_logloss: 0.560742
[1667]	valid_0's binary_logloss: 0.56073
[1668]	valid_0's binary_logloss: 0.560735
[1669]	valid_0's binary_logloss: 0.560724
[1670]	valid_0's binary_logloss: 0.560744
[1671]	valid_0's binary_logloss: 0.560687
[1672]	valid_0's binary_logloss: 0.560652
[1673]	valid_0's binary_logloss: 0.560642
[1674]	valid_0's binary_logloss: 0.560697
[1675]	valid_0's binary_logloss: 0.560713
[1676]	valid_0's binary_logloss: 0.560754
[1677]	valid_0's binary_logloss: 0.560763
[1678]	valid_0's binary_logloss: 0.560827
[1679]	valid_0's binary_logloss: 0.

[1868]	valid_0's binary_logloss: 0.560832
[1869]	valid_0's binary_logloss: 0.560826
[1870]	valid_0's binary_logloss: 0.560834
[1871]	valid_0's binary_logloss: 0.560843
[1872]	valid_0's binary_logloss: 0.560844
[1873]	valid_0's binary_logloss: 0.560796
[1874]	valid_0's binary_logloss: 0.560781
[1875]	valid_0's binary_logloss: 0.560748
[1876]	valid_0's binary_logloss: 0.560784
[1877]	valid_0's binary_logloss: 0.560798
[1878]	valid_0's binary_logloss: 0.56079
[1879]	valid_0's binary_logloss: 0.560768
[1880]	valid_0's binary_logloss: 0.560765
[1881]	valid_0's binary_logloss: 0.560754
[1882]	valid_0's binary_logloss: 0.560759
[1883]	valid_0's binary_logloss: 0.560774
[1884]	valid_0's binary_logloss: 0.560797
[1885]	valid_0's binary_logloss: 0.56077
[1886]	valid_0's binary_logloss: 0.560815
[1887]	valid_0's binary_logloss: 0.560844
[1888]	valid_0's binary_logloss: 0.56087
[1889]	valid_0's binary_logloss: 0.560906
[1890]	valid_0's binary_logloss: 0.560911
[1891]	valid_0's binary_logloss: 0.56

### DeepFM

#### 加载数据

In [1]:
import pickle

with open('lgb_values.pkl', 'rb') as f:
    x_train_resam = pickle.load(f)
    y_train_resam = pickle.load(f)
    test = pickle.load(f)

In [11]:
x_train_resam.columns

Index(['creativeID', 'userID', 'positionID', 'connectionType',
       'telecomsOperator', 'adID', 'camgaignID', 'advertiserID', 'appID',
       'appPlatform', 'click_day', 'click_min', 'click_hour',
       'app_first_categories', 'app_second_categories', 'age', 'gender',
       'education', 'marriageStatus', 'haveBaby', 'hometown_province',
       'hometown_city', 'residence_province', 'residence_city', 'sitesetID',
       'positionType'],
      dtype='object')

#### 处理数据

In [13]:
feature_dict = {}
total_feature = 0

for col in x_train_resam.columns:
    unique_val = x_train_resam[col].unique()
    feature_dict[col] = dict(zip(unique_val, range(total_feature, total_feature + len(unique_val))))
    total_feature += len(unique_val)

feature_value = x_train_resam.copy()
feature_index = x_train_resam.copy()

for col in x_train_resam.columns:
    feature_value[col] = 1
    feature_index[col] = feature_index[col].map(feature_dict[col])

#### 定义参数

In [21]:
import tensorflow as tf
import numpy as np
import pandas as pd

dfm_params = {
    'use_fm': True,
    'use_deep': True,
    'embedding_size': 8,
    'dropout_fm': [1.0, 1.0],
    'deep_layers': [32, 32],
    'dropout_deep': [0.5, 0.5, 0.5],
    'deep_layer_activation': tf.nn.relu,
    'opoch': 30,
    'bach_size': 1024,
    'learning_rate': 0.001,
    'optimizer': 'adam',
    'batch_norm_dacay': 0.995,
    'l2_reg': 0.01,
    'verbose': True,
    'eval_metric': 'logloss',
    'random_seed': 100
}
dfm_params['feature_size'] = total_feature
dfm_params['field_size'] = len(x_train_resam.columns)

with tf.device('/gpu:0'):
    feat_index = tf.placeholder(tf.int32, shape = [None, None], name = 'feat_index')
    feat_value = tf.placeholder(tf.float32, shape = [None, None], name = 'feat_value')

    label = tf.placeholder(tf.float32, shape = [None, 1], name = 'label')

    weights = dict()

    ##embeddings
    weights['feature_embeddings'] = tf.Variable(
        tf.random_normal([dfm_params['feature_size'], dfm_params['embedding_size']], 0.0, 0.01), name = 'feature_embeddings')
    weights['feature_bias'] = tf.Variable(
        tf.random_normal([dfm_params['feature_size'], 1], 0.0, 0.01), name = 'feature_bias')###weights[''feature_bias]是一次项的权重

    ##deep layers
    num_layer = len(dfm_params['deep_layers'])
    input_size = dfm_params['field_size'] * dfm_params['embedding_size']
    glorot = np.sqrt(2.0 / (input_size + dfm_params['deep_layers'][0]))

    weights['layer_0'] = tf.Variable(
        np.random.normal(loc = 0, scale = glorot, size = (input_size, dfm_params['deep_layers'][0])), dtype = np.float32)
    weights['bias_0'] = tf.Variable(
        np.random.normal(loc = 0, scale = glorot, size = (1, dfm_params['deep_layers'][0])), dtype = np.float32)

    for i in range(1, num_layer):
        glorot = np.sqrt(2.0 / (dfm_params['deep_layers'][i - 1] + dfm_params['deep_layers'][i]))
        weights['layer_%d'%(i)] = tf.Variable(
            np.random.normal(loc = 0, scale = glorot, size = (dfm_params['deep_layers'][i - 1], dfm_params['deep_layers'][i])), dtype = np.float32)
        weights['bias_%d'%(i)] = tf.Variable(
            np.random.normal(loc = 0, scale = glorot, size = (1, dfm_params['deep_layers'][i])), dtype = np.float32)

    ##concat layer
    if dfm_params['use_fm'] and dfm_params['use_deep']:
        input_size = dfm_params['field_size'] + dfm_params['embedding_size'] + dfm_params['deep_layers'][-1]
    elif dfm_params['use_fm']:
        input_size = dfm_params['field_size'] + dfm_params['embedding_size']
    else:    
        input_size = dfm_params['deep_layers'][-1]

    glorot = np.sqrt(2.0 / (input_size + 1))
    weights['concat_projection'] = tf.Variable(np.random.normal(loc = 0, scale = glorot, size = [input_size, 1]), dtype = np.float32)
    weights['concat_bias'] = tf.Variable(tf.constant(0.01), dtype = np.float32)

#### 构造模型

In [ ]:
with tf.device('/gpu:0'):
##embeddings
    embeddings = tf.nn.embedding_lookup(weights['feature_embeddings'], feat_index)
    reshaped_feat_value = tf.reshape(feat_value, shape = [-1, dfm_params['field_size'], 1])
    print(reshaped_feat_value)
    embeddings = tf.multiply(embeddings, reshaped_feat_value)

    ##fm
    fm_first_order = tf.nn.embedding_lookup(weights['feature_bias'], feat_index)
    fm_first_order = tf.reduce_sum(tf.multiply(fm_first_order, reshaped_feat_value), 2)

    summed_features_emb = tf.reduce_sum(embeddings, 1)
    summed_features_emb_square = tf.square(summed_features_emb)

    squared_features_emb = tf.square(embeddings)
    squared_features_emb_sum = tf.reduce_sum(squared_features_emb, 1)

    fm_second_order = 0.5 * tf.subtract(summed_features_emb_square, squared_features_emb_sum)

    ##deep
    y_deep = tf.reshape(embeddings, 
                        shape = [-1, dfm_params['field_size'] * dfm_params['embedding_size']])

    for i in range(len(dfm_params['deep_layers'])):
        y_deep = tf.add(tf.matmul(y_deep, weights['layer_%d'%(i)]), weights['bias_%d'%(i)])
        y_deep = tf.nn.relu(y_deep)

    ##final layer
    if dfm_params['use_fm'] and dfm_params['use_deep']:
        concat_input = tf.concat([fm_first_order, fm_second_order, y_deep], axis = 1)
    elif dfm_params['use_fm']:
        concat_input = tf.concat([fm_first_order, fm_second_order], axis = 1)
    else:
        concat_input = y_deep

    out = tf.nn.sigmoid(tf.add(tf.matmul(concat_input, 
                                         weights['concat_projection']), weights['concat_bias']))

    loss = tf.losses.log_loss(tf.reshape(label, (-1,1)), out)
    optimizer = tf.train.AdamOptimizer(learning_rate = dfm_params['learning_rate'], 
                                       beta1 = 0.9, beta2 = 0.999, epsilon = 1e-8).minimize(loss)

#### 训练模型

In [ ]:
with tf.Session(config = tf.ConfigProto(log_device_placement = True)) as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(200):
        epoch_loss,_ =sess.run([loss, optimizer], feed_dict = {feat_index: feature_index, feat_value: feature_value, label: y_train_resam})
        print('epoch%s ,loss is %s'%(str(i), str(epoch_loss)))